In [46]:
import pandas as pd

fh = "../files/picklist/candidates-sierra-export-clean-2024-09-15.csv"

In [47]:
df = pd.read_csv(
    fh,
    delimiter="\t",
    encoding="utf-8",
    engine="python",
    keep_default_na=False,
    quotechar="@",
    names=[
        "iNumber", "iCode2", "iType", "iLocation", "iStatus", "iMessage", "iOpacMessage",
        "iAgency", "iBarcode", "iUnit", "iCallNum", "iVolume", "iIntNote", "Message", "oversized",
        "DateType", "Date1", "Date2", "title", "pubInfo", "size", "bCallNum", "bStaffCallNum"])

In [48]:
locs = sorted(df["iLocation"].unique())
locs_df = pd.DataFrame(dict(location=locs))
locs_df.to_csv("../files/picklist/locations.csv", index=False)

In [49]:
iTypes = sorted(df["iType"].unique())
iTypes_df = pd.DataFrame(dict(item_type=iTypes))
iTypes_df.to_csv("../files/picklist/item_types.csv", index=False)

In [50]:
def determine_location(row):
    if row["iLocation"].startswith("rc"):
        return "ReCAP"
    elif row["iLocation"].startswith("ma") and row["iLocation"][3] == "9":
        return "SASB-M2"
    elif row["iLocation"].startswith("ma") and row["iLocation"].endswith("82"):
        return "SASB-M1"
    elif row["iLocation"].startswith("ma"):
        return "SASB"
    elif row["iLocation"].startswith("sc"):
        return "SCH"
    elif row["iLocation"].startswith("pa"):
        return "LPA"
    elif row["iLocation"].startswith("bu"):
        return "SNFL"

In [51]:
def will_not_send(row):
    if not row["iLocation"].startswith(("bu", "ma", "pa", "rc", "sc", "ls")):
        return "TRUE"
    elif int(row["iType"]) > 101:
        return "TRUE"
    elif int(row["iType"]) in (
        6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
        21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 41, 42, 43, 50, 51, 52, 53, 61, 66, 68, 70, 72,
        73, 74, 75, 76, 79):
        return "TRUE"
    elif "google" in row["iIntNote"].lower():
        return "TRUE"
    elif "poor condition" in row["iIntNote"].lower():
        return "TRUE"
    elif row["oversized"] == "False":
        return "TRUE"
    elif len(row["iBarcode"]) != 14:
        return "TRUE"
    else:
        return "FALSE"
    

In [52]:
df["location"] = df.apply(lambda row: determine_location(row), axis=1)

In [53]:
# total SASB items
all_sasb_df = df[~(df["location"].isnull()) & (df["location"].str.startswith("SASB"))]

In [54]:
all_sasb_df.shape[0]

292241

In [55]:
df["will_not_send"] = df.apply(lambda row: will_not_send(row), axis=1)

In [56]:
all_sasb_without_ineligible_df = df[~(df["location"].isnull()) & (df["location"].str.startswith("SASB") & (df["will_not_send"] == "FALSE"))]
print(all_sasb_without_ineligible_df.shape[0])

223249


In [57]:
df_sasb = df[(df["location"] == "SASB") & (df["will_not_send"] == "FALSE")]
gdf_sasb = df_sasb.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "iVolume", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_sasb.to_csv("../files/picklist/sasb.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "iVolume", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [71]:
df_sasb.shape[0]

11774

In [58]:
df_m1 = df[(df["location"] == "SASB-M1") & (df["will_not_send"] == "FALSE")]
gdf_m1 = df_m1.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))  
gdf_m1.to_csv("../files/picklist/sasb-m1.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]) 

In [72]:
df_m1.shape[0]

30421

In [59]:
df_m2 = df[(df["location"] == "SASB-M2") & (df["will_not_send"] == "FALSE")]
df_m2.to_csv("../files/picklist/sasb-m2-detailed.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])
df_m2.to_csv("../files/picklist/sasb-m2-barcodes.csv", index=False, sep="\t", columns=["iBarcode"])

In [73]:
df_m2.shape[0]

181054

In [60]:
df_recap = df[(df["location"] == "ReCAP") & (df["will_not_send"] == "FALSE")]
df_recap.to_csv("../files/picklist/recap.csv", index=False, sep="\t", columns=["iBarcode"])

In [75]:
df_recap.shape[0]

857376

In [61]:
df_lpa = df[(df["location"] == "LPA") & (df["will_not_send"] == "FALSE")]
gdf_lpa = df_lpa.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_lpa.to_csv("../files/picklist/lpa.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [76]:
df_lpa.shape[0]

1065

In [62]:
df_sch = df[(df["location"] == "SCH") & (df["will_not_send"] == "FALSE")]
gdf_sch = df_sch.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_sch.to_csv("../files/picklist/sch.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [77]:
df_sch.shape[0]

17068

In [63]:
df_snfl = df[(df["location"] == "SNFL") & (df["will_not_send"] == "FALSE")]
gdf_snfl = df_snfl.groupby("iLocation")[["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"]].apply(lambda x: x.sort_values("iCallNum"))
gdf_snfl.to_csv("../files/picklist/snfl.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [78]:
df_snfl.shape[0]

18

In [64]:
df_will_not_send = df[df["will_not_send"] == "TRUE"]
df_will_not_send.to_csv("../files/picklist/will_not_send.csv", index=False, sep="\t", columns=["iBarcode"])

In [79]:
df_will_not_send.shape[0]

134297

In [65]:
df_sch[df_sch["iCallNum"] == ""].to_csv("../files/picklist/sch-missing-callnum.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

In [66]:
# invalid barcodes
df_invalid_barcodes = df[(df["iBarcode"].str.len() != 14)]
print(df_invalid_barcodes.shape[0])
df_invalid_barcodes.to_csv("../files/picklist/invalid_barcodes.csv", index=False, sep="\t", columns=["iNumber", "iLocation", "iBarcode", "iCallNum", "title", "pubInfo", "bCallNum", "bStaffCallNum"])

95


In [69]:
# make sure the final count is correct
print(f"total # of items: {df.shape[0]}")
locs_total = df_sasb.shape[0] + df_m1.shape[0] + df_m2.shape[0] + df_recap.shape[0] + df_lpa.shape[0] + df_sch.shape[0] + df_snfl.shape[0] + df_will_not_send.shape[0]
print(f"sum of items in all locations: {locs_total}")

total # of items: 1233074
sum of items in all locations: 1233073


In [70]:
assert df.shape[0] == locs_total

AssertionError: 